# E15. seq2seq 단어 단위 번역기 만들기

In [1]:
import pandas as pd
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

1 Physical GPUs, 1 Logical GPUs


In [2]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model
import re
print('⏳')

⏳


__훈련 데이터 샘플 33000개만 사용하기__

In [3]:
import os
file_path = os.getenv('HOME')+'/aiffel/e/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')

lines = lines[['eng','fra']][:33000]
print('전체 샘플의 수 :',len(lines))
lines.sample(5) #샘플 5개 출력

전체 샘플의 수 : 33000


,eng,fra
31419,She baked me a cake.,Elle me concocta un gâteau.
24656,I should be in bed.,Je devrais être au lit.
17892,You're unethical.,Tu es immorale.
25987,She's a sweet girl.,C'est une gentille fille.
16046,No one ever came.,Personne ne vint jamais.


## Step 1. 정제, 정규화, 전처리 (영어, 프랑스어 모두!)

In [19]:
def preprocess_sentence(sentence):
    # 2. 소문자로 바꾸기
    sentence = sentence.lower().strip()
    
    # 1. 구두점을 단어와 분리
    sentence = re.sub(r"(['?.!,-])",r" \1 ", sentence)
    
    sentence = sentence.strip().split()
    return sentence

In [20]:
sentence = "Hi my name's Slim Shady."
sentence = preprocess_sentence(sentence)
print(sentence)

['hi', 'my', 'name', "'", 's', 'slim', 'shady', '.']


In [21]:
lines.describe()

,eng,fra
count,33000,33000
unique,20971,30176
top,you ' re the teacher .,<sos> merci bien . <eos>
freq,26,9


## Step 2. 디코더의 문장에 시작 토큰과 종료 토큰을 넣어주세요.

In [22]:
lines.eng = lines.eng.apply(lambda x: preprocess_sentence(x))
lines.fra = lines.fra.apply(lambda x: preprocess_sentence(x))
lines.sample(5)

,eng,fra
23280,"[don, ', t, do, it, for, me, .]","[<sos>, ne, le, fais, pas, pour, moi, ., <eos>]"
6112,"[it, can, ', t, wait, .]","[<sos>, ça, ne, peut, pas, attendre, ., <eos>]"
20236,"[it, rained, heavily, .]","[<sos>, il, a, plu, énormément, ., <eos>]"
14495,"[how, good, are, you, ?]","[<sos>, jusqu, ', à, quel, point, êtes, -, vou..."
24670,"[i, sliced, the, apple, .]","[<sos>, j, ', ai, coupé, la, pomme, ., <eos>]"


sos_token = '<sos>'
eos_token = '<eos>'

lines.fra = lines.fra.apply(lambda x: '<sos> ' + x + ' <eos>')
lines.sample(5)

## Step 3. 케라스의 토크나이저로 텍스트를 숫자로 바꿔보세요.
---
tokenizer.texts_to_sequences()를 사용하여 모든 샘플에 대해서 정수 시퀀스로 변환

In [24]:
eng_tokenizer = Tokenizer(char_level=False) # 단어 단위 Tokenizer
eng_tokenizer.fit_on_texts(lines.eng)       # 33000개 eng 행에 각각 토큰화
input_text = eng_tokenizer.texts_to_sequences(lines.eng)  # 단어 -> 숫자값 인덱스 변환 저장
input_text[:3]

[[31, 1], [1135, 1], [1135, 1]]

In [25]:
fra_tokenizer = Tokenizer(char_level=False)
fra_tokenizer.fit_on_texts(lines.fra)
target_text = fra_tokenizer.texts_to_sequences(lines.fra)
target_text[:3]

[[1, 93, 13, 2], [1, 1037, 13, 2], [1, 1037, 3, 2]]

In [26]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print("영어 단어장 크기:", eng_vocab_size, "\n불어 단어장 크기", fra_vocab_size)

영어 단어장 크기: 4705 
불어 단어장 크기 8480


In [27]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 10
프랑스어 시퀀스의 최대 길이 20


__전체 통계량__

In [28]:
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 33000
영어 단어장의 크기 : 4705
프랑스어 단어장의 크기 : 8480
영어 시퀀스의 최대 길이 10
프랑스어 시퀀스의 최대 길이 20


__시작, 종료 토큰 각각 제거__

In [29]:
encoder_input = input_text
decoder_input = [[word for word in line if word != fra_tokenizer.word_index[eos_token]] for line in target_text]
decoder_target = [[word for word in line if word != fra_tokenizer.word_index[sos_token]] for line in target_text]

In [30]:
print(decoder_input[:3])
print(decoder_target[:3])

[[1, 93, 13], [1, 1037, 13], [1, 1037, 3]]
[[93, 13, 2], [1037, 13, 2], [1037, 3, 2]]


In [31]:
# padding
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding="post")
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding="post")
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding="post")
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 10)
프랑스어 입력데이터의 크기(shape) : (33000, 20)
프랑스어 출력데이터의 크기(shape) : (33000, 20)


__shuffle__

In [33]:
idx = np.arange(encoder_input.shape[0])
np.random.shuffle(idx)

encoder_input = encoder_input[idx]
decoder_input = decoder_input[idx]
decoder_target = decoder_target[idx]

In [35]:
print(encoder_input[0])
print(idx)

[ 11   2  10 448   1   0   0   0   0   0]
[13147 10953  8704 ... 23061  1515 19530]


In [36]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]


In [37]:
print('영어 학습데이터의 크기(shape) :',np.shape(encoder_input_train))
print('프랑스어 학습 입력데이터의 크기(shape) :',np.shape(decoder_input_train))
print('프랑스어 학습 출력데이터의 크기(shape) :',np.shape(decoder_target_train))

print('영어 검증데이터의 크기(shape) :',np.shape(encoder_input_test))
print('프랑스어 검증 입력데이터의 크기(shape) :',np.shape(decoder_input_test))
print('프랑스어 검증 출력데이터의 크기(shape) :',np.shape(decoder_target_test))

영어 학습데이터의 크기(shape) : (30000, 10)
프랑스어 학습 입력데이터의 크기(shape) : (30000, 20)
프랑스어 학습 출력데이터의 크기(shape) : (30000, 20)
영어 검증데이터의 크기(shape) : (3000, 10)
프랑스어 검증 입력데이터의 크기(shape) : (3000, 20)
프랑스어 검증 출력데이터의 크기(shape) : (3000, 20)


## Step 4. 임베딩 층(Embedding layer) 사용하기
---
이번에는 입력이 되는 각 단어를 임베딩 층을 사용하여 벡터화하겠습니다.

임베딩 층을 사용하는 방법과 그 설명에 대해서는 아래의 링크의

__1. 케라스 임베딩 층(Keras Embedding layer)__를 참고하세요.

[위키독스](https://wikidocs.net/33793)

실제 번역기 구현을 위해서 사용할 수 있는 인코더 코드의 예시는 다음과 같습니다.

이를 통해서 인코더와 디코더의 임베딩 층을 각각 구현해보세요.

from tensorflow.keras.layers import Input, mbedding, Masking

```python
# 인코더에서 사용할 임베딩 층 사용 예시
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(단어장의 크기, 임베딩 벡터의 차원)(encoder_inputs)
encoder_lstm = LSTM(hidden state의 크기, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
```

주의할 점은 인코더와 디코더의 임베딩 층은 서로 다른 임베딩 층을 사용해야 하지만,

디코더의 훈련 과정과 테스트 과정(예측 과정)에서의 임베딩 층은 동일해야 합니다!

In [39]:
output_size = 256

__인코더 임베딩__

In [41]:
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(eng_vocab_size, output_size, input_length = max_eng_seq_len)(encoder_inputs)
# padding값은 연산하지 않는다
enc_masking = Masking(mask_value=0.0)(enc_emb)
encoder_lstm = LSTM(units=256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)
encoder_states = [state_h, state_c]

__디코더 임베딩__

In [ ]:
decoder_inputs = Input(shape=(None, ))
dec_emb_layer = Embedding(fra_vocab_size, output_dim)
dec_emb = dec_emb_layer(decoder_inputs)
dec_masking = Masking(mask_value=0.0)(dec_emb)
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_masking, initial_state = encoder_states)

In [ ]:
decoder_softmax_layer = Dense(fra_vocab_size, activation="softmax")
decoder_outputs = decoder_softmax_layer(decoder_outputs)

## Step 5. 모델 구현하기
---
글자 단위 번역기에서 구현한 모델을 참고로 단어 단위 번역기의 모델을 완성시켜보세요!

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy")
mode.summary()

In [ ]:
model.summary()

In [ ]:
model.fit(x=[encoder_input_train, decoder_input_train], y = decoder_target_train, \
         validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
         batch_size=64, epochs=20)

In [ ]:
# 입력 텐서 생성.
encoder_inputs = Input(shape=(None, eng_vocab_size))
# hidden size가 256인 인코더의 LSTM 셀 생성
encoder_lstm = LSTM(units = 256, return_state = True)
# 디코더로 전달할 hidden state, cell state를 리턴. encoder_outputs은 여기서는 불필요.
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# hidden state와 cell state를 다음 time step으로 전달하기 위해서 별도 저장.
encoder_states = [state_h, state_c]

In [ ]:
# 입력 텐서 생성.
decoder_inputs = Input(shape=(None, fra_vocab_size))
# hidden size가 256인 인코더의 LSTM 셀 생성
decoder_lstm = LSTM(units = 256, return_sequences = True, return_state=True)
# decoder_outputs는 모든 time step의 hidden state
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state = encoder_states)

In [ ]:
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=128, epochs=50)

In [ ]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

In [ ]:
# 이전 time step의 hidden state를 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
# 이전 time step의 cell state를 저장하는 텐서
decoder_state_input_c = Input(shape=(256,))
# 이전 time step의 hidden state와 cell state를 하나의 변수에 저장
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# decoder_states_inputs를 현재 time step의 초기 상태로 사용.
# 구체적인 동작 자체는 def decode_sequence()에 구현.
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state = decoder_states_inputs)
# 현재 time step의 hidden state와 cell state를 하나의 변수에 저장.
decoder_states = [state_h, state_c]

In [ ]:
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)
decoder_model.summary()

In [ ]:
eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

# plot_model(model)

In [ ]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, fra_vocab_size))
    target_seq[0, 0, fra2idx['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, fra_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [ ]:
import numpy as np
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스 (자유롭게 선택해 보세요)
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.eng[seq_index])
    print('정답 문장:', lines.fra[seq_index][1:len(lines.fra[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

## Step 6. 모델 평가하기
---
단어 단위 번역기에 대해서 훈련 데이터의 샘플과 테스트 데이터의 샘플에 대해서 번역 문장을 만들어보고 정답 문장과 번역 문장을 비교해보세요.

# 루브릭

아래의 기준을 바탕으로 프로젝트를 평가합니다.

### 평가문항
### 상세기준
---
__1. 번역기 모델 학습에 필요한 텍스트 데이터 전처리가 잘 이루어졌다.__

구두점, 대소문자, 띄어쓰기 등 번역기 모델에 요구되는 전처리가 정상적으로 진행되었다.

---
__2. seq2seq 기반의 번역기 모델이 정상적으로 구동된다.__

seq2seq 모델 훈련결과 validation loss가 안정적으로 떨어지면서 학습이 진행됨이 확인되었다.

---
__3. 테스트 결과 의미가 통하는 수준의 번역문이 생성되었다.__

테스트용 디코더 모델이 정상적으로 만들어져서, 정답과 어느정도 유사한 프랑스어 번역이 진행됨을 확인하였다.